Trabalho - Métodos de Investigação Operacional

In [2]:
from pulp import *

In [3]:
# Criar o problema de maximização
prob = LpProblem("Plano_Ótimo_de_Financiamento", LpMaximize)

In [4]:
# ====== VARIÁVEIS DE DECISÃO ======
# Variáveis para linha de crédito (L_t)
L = {t: LpVariable(f"L_{t}", lowBound=0, upBound=100) for t in range(1, 7)}

# Variáveis para papel comercial emitido nos dois primeiros meses (P_{t3})
P = {t: LpVariable(f"P_{t}_3", lowBound=0) for t in range(1, 3)}

# Variáveis para investimento (I_t)
I = {t: LpVariable(f"I_{t}", lowBound=0) for t in range(1, 7)}

# Variáveis para saldo de caixa (S_t)
S = {t: LpVariable(f"S_{t}", lowBound=0) for t in range(1, 7)}

In [5]:
# ====== DADOS DO PROBLEMA ======
fluxo_caixa = [-100, -200, 100, -150, 550, -100]  # Fluxos de caixa mensais (em milhares de euros)

In [6]:
# ====== FUNÇÃO OBJETIVO ======
# Maximizar o saldo de caixa no final de junho
prob += S[6], "Saldo_Final_Junho"

In [7]:
# ====== RESTRIÇÕES ======
for t in range(1, 7):
    if t == 1:  # Janeiro (primeiro mês)
        prob += S[t] == fluxo_caixa[t - 1] + L[t] + P.get(t, 0) - I[t]
    elif t == 2:  # Fevereiro
        prob += S[t] == S[t - 1] + fluxo_caixa[t - 1] + L[t] + P.get(t, 0) - (1.01 * L[t - 1]) - I[t]
    elif t == 3:  # Março
        prob += S[t] == S[t - 1] + fluxo_caixa[t - 1] + L[t] - (1.01 * L[t - 1]) - I[t]
    elif t == 4:  # Abril (reembolso de P_1_3)
        prob += S[t] == S[t - 1] + fluxo_caixa[t - 1] + L[t] - (1.01 * L[t - 1]) - (1.02 * P.get(1, 0)) - I[t]
    elif t == 5:  # Maio (reembolso de P_2_3)
        prob += S[t] == S[t - 1] + fluxo_caixa[t - 1] + L[t] - (1.01 * L[t - 1]) - (1.02 * P.get(2, 0)) - I[t]
    else:  # Junho (último mês)
        prob += S[t] == S[t - 1] + fluxo_caixa[t - 1] + L[t] - (1.01 * L[t - 1]) - I[t] + (0.001 * I[t - 1])


In [8]:
# ====== RESOLVER O MODELO ======
prob.solve()

1

In [9]:
# ====== RESULTADOS ======
# Status do problema
print("Status:", LpStatus[prob.status])

# Solução ótima
print("\nSolução Ótima:")
for t in range(1, 7):
    print(f"Saldo final no mês {t}: {S[t].varValue:.2f} mil euros")
    print(f"  Linha de crédito (L_{t}): {L[t].varValue:.2f} mil euros")
    print(f"  Investimento (I_{t}): {I[t].varValue:.2f} mil euros")
    if t <= 2:
        print(f"  Papel comercial emitido (P_{t}_3): {P[t].varValue:.2f} mil euros")
    print("-" * 40)

Status: Optimal

Solução Ótima:
Saldo final no mês 1: 0.00 mil euros
  Linha de crédito (L_1): 50.98 mil euros
  Investimento (I_1): 0.00 mil euros
  Papel comercial emitido (P_1_3): 49.02 mil euros
----------------------------------------
Saldo final no mês 2: 0.00 mil euros
  Linha de crédito (L_2): 0.00 mil euros
  Investimento (I_2): 0.00 mil euros
  Papel comercial emitido (P_2_3): 251.49 mil euros
----------------------------------------
Saldo final no mês 3: 100.00 mil euros
  Linha de crédito (L_3): 0.00 mil euros
  Investimento (I_3): 0.00 mil euros
----------------------------------------
Saldo final no mês 4: 0.00 mil euros
  Linha de crédito (L_4): 100.00 mil euros
  Investimento (I_4): 0.00 mil euros
----------------------------------------
Saldo final no mês 5: 192.48 mil euros
  Linha de crédito (L_5): 0.00 mil euros
  Investimento (I_5): 0.00 mil euros
----------------------------------------
Saldo final no mês 6: 192.48 mil euros
  Linha de crédito (L_6): 100.00 mil eu

In [10]:
# Valor da função objetivo
print("\nSaldo final em junho (função objetivo):", S[6].varValue, "mil euros")


Saldo final em junho (função objetivo): 192.48 mil euros


In [11]:
# Variáveis de folga
print("\nVariáveis de Folga:")
for nome, restricao in prob.constraints.items():
    print(f"{nome}: {restricao.slack:.2f}")


Variáveis de Folga:
_C1: -0.00
_C2: -0.00
_C3: -0.00
_C4: -0.00
_C5: -0.00
_C6: -0.00


Problema Dual

In [12]:
from pulp import *

In [13]:
# Criar o problema de minimização
dual_problem = LpProblem("Problema_Dual", LpMinimize)

In [14]:
# ====== VARIÁVEIS DE DECISÃO ======
y1 = LpVariable("y1", lowBound=0, cat='Continuous')
y2 = LpVariable("y2", lowBound=0, cat='Continuous')
y3 = LpVariable("y3", lowBound=0, cat='Continuous')
y4 = LpVariable("y4", lowBound=0, cat='Continuous')
y5 = LpVariable("y5", lowBound=0, cat='Continuous')
y6 = LpVariable("y6", lowBound=0, cat='Continuous')
lambda1 = LpVariable("lambda1", lowBound=0, cat='Continuous')
lambda2 = LpVariable("lambda2", lowBound=0, cat='Continuous')

In [15]:
# Definir a função objetivo (custo mínimo)
dual_problem += y1 * 100 + y2 * 200 + y3 * 100 + y4 * 150 + y5 * 550 + y6 * 100, "Função Objetivo"


In [16]:
# Definir as restrições do problema dual
dual_problem += y1 + y2 + y3 + y4 + y5 + y6 >= 1, "Restrição de Fluxo de Caixa"
dual_problem += lambda1 >= 0, "Restrição de Linha de Crédito"
dual_problem += lambda2 >= 0, "Restrição de Papel Comercial"
dual_problem += y1*(-100) + y2*(-200) + y3*100 + y4*(-150) + y5*550 + y6*(-100) >= 0, "Restrição de Balanço de Caixa"


In [17]:
# Resolver o problema
dual_problem.solve()

1

In [18]:
# Exibir os resultados
print(f"Solução ótima do problema dual:")
print(f"y1 = {y1.varValue}")
print(f"y2 = {y2.varValue}")
print(f"y3 = {y3.varValue}")
print(f"y4 = {y4.varValue}")
print(f"y5 = {y5.varValue}")
print(f"y6 = {y6.varValue}")
print(f"lambda1 = {lambda1.varValue}")
print(f"lambda2 = {lambda2.varValue}")

Solução ótima do problema dual:
y1 = 0.5
y2 = 0.0
y3 = 0.5
y4 = 0.0
y5 = 0.0
y6 = 0.0
lambda1 = 0.0
lambda2 = 0.0


In [ ]:
# Exibir as variáveis de folga 
for name, constraint in dual_problem.constraints.items():
    print(f"{name}: {constraint.slack}")

Restrição_de_Fluxo_de_Caixa: -0.0
Restrição_de_Linha_de_Crédito: -0.0
Restrição_de_Papel_Comercial: -0.0
Restrição_de_Balanço_de_Caixa: -0.0


In [20]:
# Exibindo os preços sombra das restrições
for name, constraint in prob.constraints.items():
    print(f"Preço sombra da restrição {name}: {constraint.pi}")


Preço sombra da restrição _C1: 1.0302
Preço sombra da restrição _C2: 1.02
Preço sombra da restrição _C3: 1.01
Preço sombra da restrição _C4: 1.01
Preço sombra da restrição _C5: 1.0
Preço sombra da restrição _C6: 1.0
